In [1]:
! pip install pyspark pandas numpy

In [2]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
# from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder, StandardScaler
# from pyspark.ml.evaluation import MulticlassClassificationEvaluator
# from pyspark.ml.regression import LinearRegression, DecisionTreeRegressor, RandomForestRegressor, GBTRegressor
# from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import col
# import shutil
from pyspark.sql.types import *
import pyspark.sql.functions as Spark_f
from pyspark import SparkConf
from pyspark.sql.functions import format_number
import pandas, numpy
import gc

In [3]:
# spark = SparkSession.builder.appName("PySparkAmazonReviewsTestSet").getOrCreate()
conf = SparkConf() \
    .setMaster("local[10]") \
    .set("spark.driver.memory", "2g") \
    .set("spark.executor.memory", "4g") \
    .set("spark.shuffle.spill", "true") \
    .set("spark.network.timeout", "60s") \
    .set("spark.executor.extraJavaOptions", "-XX:+UseG1GC -XX:InitiatingHeapOccupancyPercent=35 -XX:G1ReservePercent=20") \
    .set("spark.sql.legacy.timeParserPolicy","LEGACY")

spark = SparkSession.builder.config(conf=conf).appName("PySparkAmazonReviewsTestSet").getOrCreate()

24/11/28 03:12:24 WARN Utils: Your hostname, NBM-WXX9-7572c0be resolves to a loopback address: 127.0.1.1; using 192.168.88.79 instead (on interface wlp1s0)
24/11/28 03:12:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/28 03:12:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark

In [ ]:
pd_df1 = pandas.read_csv("/dataset/test/input/books_data.csv", dtype={'Title': str, 'description': str, 'authors':str,'image':str,'previewLink':str,'publisher':str,'publishedDate':str,'infoLink':str,'categories':str,'ratingsCount':float})
pd_df2 = pandas.read_csv("/dataset/test/input/Books_rating.csv")
pd_df1.head(10)

24/11/28 03:12:38 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


,Title,description,authors,image,previewLink,publisher,publishedDate,infoLink,categories,ratingsCount
0,Its Only Art If Its Well Hung!,NaN,['Julie Strain'],http://books.google.com/books/content?id=DykPA...,http://books.google.nl/books?id=DykPAAAACAAJ&d...,NaN,1996,http://books.google.nl/books?id=DykPAAAACAAJ&d...,['Comics & Graphic Novels'],NaN
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],http://books.google.com/books/content?id=IjvHQ...,http://books.google.nl/books?id=IjvHQsCn_pgC&p...,A&C Black,2005-01-01,http://books.google.nl/books?id=IjvHQsCn_pgC&d...,['Biography & Autobiography'],NaN
2,Wonderful Worship in Smaller Churches,This resource includes twelve principles in un...,['David R. Ray'],http://books.google.com/books/content?id=2tsDA...,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,NaN,2000,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,['Religion'],NaN
3,Whispers of the Wicked Saints,Julia Thomas finds her life spinning out of co...,['Veronica Haddon'],http://books.google.com/books/content?id=aRSIg...,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,iUniverse,2005-02,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,['Fiction'],NaN
4,"Nation Dance: Religion, Identity and Cultural ...",NaN,['Edward Long'],NaN,http://books.google.nl/books?id=399SPgAACAAJ&d...,NaN,2003-03-01,http://books.google.nl/books?id=399SPgAACAAJ&d...,NaN,NaN
5,The Church of Christ: A Biblical Ecclesiology ...,In The Church of Christ: A Biblical Ecclesiolo...,['Everett Ferguson'],http://books.google.com/books/content?id=kVqRa...,http://books.google.nl/books?id=kVqRaiPlx88C&p...,Wm. B. Eerdmans Publishing,1996,http://books.google.nl/books?id=kVqRaiPlx88C&d...,['Religion'],5.0
6,The Overbury affair (Avon),NaN,['Miriam Allen De Ford'],NaN,http://books.google.nl/books?id=mHLTngEACAAJ&d...,NaN,1960,http://books.google.nl/books?id=mHLTngEACAAJ&d...,NaN,NaN
7,A Walk in the Woods: a Play in Two Acts,NaN,['Lee Blessing'],NaN,http://books.google.nl/books?id=6HDOwAEACAAJ&d...,NaN,1988,http://books.google.nl/books?id=6HDOwAEACAAJ&d...,NaN,3.0
8,Saint Hyacinth of Poland,The story for children 10 and up of St. Hyacin...,['Mary Fabyan Windeatt'],http://books.google.com/books/content?id=lmLqA...,http://books.google.nl/books?id=lmLqAAAACAAJ&d...,Tan Books & Pub,2009-01-01,http://books.google.nl/books?id=lmLqAAAACAAJ&d...,['Biography & Autobiography'],NaN
9,Rising Sons and Daughters: Life Among Japan's ...,Wardell recalls his experience as a foreign st...,['Steven Wardell'],NaN,http://books.google.nl/books?id=rbLZugEACAAJ&d...,Plympton PressIntl,1995,http://books.google.nl/books?id=rbLZugEACAAJ&d...,['Social Science'],NaN


In [7]:
print(pd_df1.dtypes)
pd_df2.dtypes

Title             object
description       object
authors           object
image             object
previewLink       object
publisher         object
publishedDate     object
infoLink          object
categories        object
ratingsCount     float64
dtype: object


Id                     object
Title                  object
Price                 float64
User_id                object
profileName            object
review/helpfulness     object
review/score          float64
review/time             int64
review/summary         object
review/text            object
dtype: object

In [8]:
pd_df1.to_parquet('/dataset/test/df1.parquet')
pd_df2.to_parquet('/dataset/test/df2.parquet')
del pd_df1,pd_df2
gc.collect() # очистка памяти

0

In [9]:
df1 = spark.read.parquet('/dataset/test/df1.parquet')
df2 = spark.read.parquet('/dataset/test/df2.parquet')
df1.show()
df2.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+------------+
|               Title|         description|             authors|               image|         previewLink|           publisher|publishedDate|            infoLink|          categories|ratingsCount|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+------------+
|Its Only Art If I...|                NULL|    ['Julie Strain']|http://books.goog...|http://books.goog...|                NULL|         1996|http://books.goog...|['Comics & Graphi...|        NULL|
|Dr. Seuss: Americ...|Philip Nel takes ...|      ['Philip Nel']|http://books.goog...|http://books.goog...|           A&C Black|   2005-01-01|http://books.goog...|['Biography & Aut...|        NULL|
|Wonderful Wors

In [10]:
print(df1.dtypes)
df2.dtypes

[('Title', 'string'), ('description', 'string'), ('authors', 'string'), ('image', 'string'), ('previewLink', 'string'), ('publisher', 'string'), ('publishedDate', 'string'), ('infoLink', 'string'), ('categories', 'string'), ('ratingsCount', 'double')]


[('Id', 'string'),
 ('Title', 'string'),
 ('Price', 'double'),
 ('User_id', 'string'),
 ('profileName', 'string'),
 ('review/helpfulness', 'string'),
 ('review/score', 'double'),
 ('review/time', 'bigint'),
 ('review/summary', 'string'),
 ('review/text', 'string')]

In [11]:
book_meta = df1.drop("ratingsCount").withColumns({"publishedDate":Spark_f.to_date(col("publishedDate")),"authors":Spark_f.split(Spark_f.regexp_replace(Spark_f.col("authors"), r"[\[\]']", ""), ","),"categories":Spark_f.split(Spark_f.regexp_replace(Spark_f.col("categories"), r"[\[\]']", ""), ",")})
book_meta.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+
|               Title|         description|             authors|               image|         previewLink|           publisher|publishedDate|            infoLink|          categories|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+
|Its Only Art If I...|                NULL|      [Julie Strain]|http://books.goog...|http://books.goog...|                NULL|   1996-01-01|http://books.goog...|[Comics & Graphic...|
|Dr. Seuss: Americ...|Philip Nel takes ...|        [Philip Nel]|http://books.goog...|http://books.goog...|           A&C Black|   2005-01-01|http://books.goog...|[Biography & Auto...|
|Wonderful Worship...|This resource inc...|      [David R. Ray]|http://books.goo

In [12]:
review_helpfulness = Spark_f.split(df2['review/helpfulness'],'/')
book_review = df2.withColumns({"review/time":Spark_f.to_date(Spark_f.from_unixtime(col("review/time").cast("long"))),
                               'helpful_votes':review_helpfulness.getItem(0).cast('int'),
                               'total_votes':review_helpfulness.getItem(1).cast('int')}).drop('Price','review/helpfulness')
book_review.show()


+----------+--------------------+--------------+--------------------+------------+-----------+--------------------+--------------------+-------------+-----------+
|        Id|               Title|       User_id|         profileName|review/score|review/time|      review/summary|         review/text|helpful_votes|total_votes|
+----------+--------------------+--------------+--------------------+------------+-----------+--------------------+--------------------+-------------+-----------+
|1882931173|Its Only Art If I...| AVCGYZL8FQQTD|Jim of Oz "jim-of...|         4.0| 1999-10-23|Nice collection o...|This is only for ...|            7|          7|
|0826414346|Dr. Seuss: Americ...|A30TK6U7DNS82R|       Kevin Killian|         5.0| 2004-09-21|   Really Enjoyed It|I don't care much...|           10|         10|
|0826414346|Dr. Seuss: Americ...|A3UH4UZ4RSVO82|        John Granger|         5.0| 2004-03-09|Essential for eve...|If people become ...|           10|         11|
|0826414346|Dr. Seuss:

In [13]:
book_review = book_review.filter(Spark_f.isnotnull(book_review.User_id))

In [14]:
book_meta = book_meta.join(book_review.groupBy("Title").agg(Spark_f.count("*").alias("popularity")),on="Title", how="left").sort(Spark_f.desc("popularity")).limit(2000) # для последующих фильтров
book_meta.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-------------------+----------+
|               Title|         description|             authors|               image|         previewLink|           publisher|publishedDate|            infoLink|         categories|popularity|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-------------------+----------+
|          The Hobbit|Celebrating 75 ye...|  [J. R. R. Tolkien]|http://books.goog...|http://books.goog...|       Mariner Books|   2012-01-01|http://books.goog...| [Juvenile Fiction]|     18237|
| Pride and Prejudice|In early nineteen...|       [Jane Austen]|http://books.goog...|http://books.goog...| Courier Corporation|   1995-01-01|http://books.goog...|          [Fiction]|     17969|
|      Atlas Shrugged|        

In [15]:
book_meta.select([Spark_f.count(Spark_f.when(Spark_f.isnull(c), c)).alias(c) for c in book_meta.columns]).show()

+-----+-----------+-------+-----+-----------+---------+-------------+--------+----------+----------+
|Title|description|authors|image|previewLink|publisher|publishedDate|infoLink|categories|popularity|
+-----+-----------+-------+-----+-----------+---------+-------------+--------+----------+----------+
|    0|        370|    232|  311|        194|      452|          202|     194|       348|         0|
+-----+-----------+-------+-----+-----------+---------+-------------+--------+----------+----------+



In [16]:
book_meta = book_meta.filter(Spark_f.isnotnull(book_meta['previewLink'])).limit(1000)
book_meta.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-------------------+----------+
|               Title|         description|             authors|               image|         previewLink|           publisher|publishedDate|            infoLink|         categories|popularity|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-------------------+----------+
|          The Hobbit|Celebrating 75 ye...|  [J. R. R. Tolkien]|http://books.goog...|http://books.goog...|       Mariner Books|   2012-01-01|http://books.goog...| [Juvenile Fiction]|     18237|
| Pride and Prejudice|In early nineteen...|       [Jane Austen]|http://books.goog...|http://books.goog...| Courier Corporation|   1995-01-01|http://books.goog...|          [Fiction]|     17969|
|   Wuthering Heights|Wutherin

In [17]:
book_review.select([Spark_f.count(Spark_f.when(Spark_f.isnull(c), c)).alias(c) for c in book_review.columns]).show()

+---+-----+-------+-----------+------------+-----------+--------------+-----------+-------------+-----------+
| Id|Title|User_id|profileName|review/score|review/time|review/summary|review/text|helpful_votes|total_votes|
+---+-----+-------+-----------+------------+-----------+--------------+-----------+-------------+-----------+
|  0|  195|      0|        118|           0|          0|           397|          1|            0|          0|
+---+-----+-------+-----------+------------+-----------+--------------+-----------+-------------+-----------+



In [18]:
book_review = book_review.join(book_meta,on='Title',how='leftsemi').sort(Spark_f.desc(book_review.helpful_votes))
print(book_review.count())
book_review.show()

705707


+--------------------+----------+--------------+--------------------+------------+-----------+--------------------+--------------------+-------------+-----------+
|               Title|        Id|       User_id|         profileName|review/score|review/time|      review/summary|         review/text|helpful_votes|total_votes|
+--------------------+----------+--------------+--------------------+------------+-----------+--------------------+--------------------+-------------+-----------+
|  Rich Dad, Poor Dad|B000GSDG8E| AS9CUR7C69N6W|"korak@evilemail....|         1.0| 2000-10-15|Not worth the mon...|I know this book ...|         3435|       3755|
|Liberal Fascism: ...|0385511841|A2LEVKA3U6CXX7|David McCune "sel...|         5.0| 2008-01-08|"I ask you to jud...|And boy, does Jon...|         2927|       3616|
|           NEW EARTH|B000N6R4AA| ARK5SAGVLW3FY|        bunnyrabbit4|         1.0| 2008-04-14| Buddhism repackaged|This is an intere...|         2425|       2948|
|Unfit for Command...|

In [19]:
book_review.groupBy(book_review.User_id).count().sort(Spark_f.desc('count')).show()

+--------------+-----+
|       User_id|count|
+--------------+-----+
|A1D2C0WDCSHUWZ| 1977|
|A20EEWWSFMZ1PN|  749|
|A1EKTLUL24HDG8|  379|
|A1X8VZWTOG8IS6|  364|
|A1K1JW1C5CUSUZ|  361|
|A1N1YEMTI9DJ86|  293|
|A3OH101U0CPUC7|  293|
|A1T17LMQABMBN5|  277|
| AHD101501WCN1|  264|
|A1G37DFO8MQW0M|  263|
| AJQ1S39GZBKUG|  259|
|A1LMBM1N4EXS5W|  258|
|A22DUZU3XVA8HA|  251|
|A3IKBHODOTYYHM|  249|
|A3QVAKVRAH657N|  243|
| AHXAPVSHPJ6OJ|  241|
|A3NI29U4P5NQF1|  230|
|A29NUB3P6YIWZG|  217|
| A4FX5YCJA630V|  217|
|A28WJUJF6D2ULA|  208|
+--------------+-----+
only showing top 20 rows



In [20]:
book_meta = book_meta.join(book_review.groupBy(book_review.Title).agg(Spark_f.avg('review/score').alias('avgScore')),on="Title", how="left")
book_meta.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+----------+------------------+
|               Title|         description|             authors|               image|         previewLink|           publisher|publishedDate|            infoLink|          categories|popularity|          avgScore|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+----------+------------------+
|          The Hobbit|Celebrating 75 ye...|  [J. R. R. Tolkien]|http://books.goog...|http://books.goog...|       Mariner Books|   2012-01-01|http://books.goog...|  [Juvenile Fiction]|     18237|4.6797718923068485|
|      Fahrenheit 451|                NULL|      [Ray Bradbury]|http://books.goog...|http://books.goog...|                NULL|   2012-01-01|htt

In [31]:
print(book_meta.count())
print(book_review.count())
book_review.groupBy(book_review.User_id).count().select(Spark_f.avg('count')).show()

1000


705707
+------------------+
|        avg(count)|
+------------------+
|3.0637755655794288|
+------------------+



In [32]:
book_meta.coalesce(1).write.mode("overwrite") \
    .option("compression", "snappy") \
    .parquet("/dataset/test/test_book_meta")
book_review.coalesce(1).write.mode("overwrite") \
    .option("compression", "snappy") \
    .parquet("/dataset/test/test_book_review")